# Spring returns to the Great Plains

Mapping Tasiyagnunpa migration

## Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace.

In [1]:
#This restore is completed in the next cell block
#%store -r

# Import libraries
import geopandas as gpd #for vector
import os
import pathlib
import pandas as pd #for tabular
import earthpy #for data management

## STEP 4: Count the number of observations in each ecosystem, during each month of 2023

Much of the data in GBIF is **crowd-sourced**. As a result, we need not
just the number of observations in each ecosystem each month – we need
to **normalize** by some measure of **sampling effort**. After all, we
wouldn’t expect the same number of observations at the North Pole as we
would in a National Park, even if there were the same number organisms.
In this case, we’re normalizing using the average number of observations
for each ecosystem and each month. This should help control for the
number of active observers in each location and time of year.

### Set up your analysis

First things first – let’s load your stored variables.

In [2]:
%store -r ecoregions_gdf gbif_gdf

### Identify the ecoregion for each observation

You can combine the ecoregions and the observations **spatially** using
a method called `.sjoin()`, which stands for spatial join.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-read"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Read More</div></div><div class="callout-body-container callout-body"><p>Check out the <a
href="https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#spatial-joins"><code>geopandas</code>
documentation on spatial joins</a> to help you figure this one out. You
can also ask your favorite LLM (Large-Language Model, like ChatGPT)</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Perform a spatial join</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Identify the correct values for the <code>how=</code> and
<code>predicate=</code> parameters of the spatial join.</li>
<li>Select only the columns you will need for your plot.</li>
<li>Run the code.</li>
</ol></div></div>

In [3]:
ecoregions_gdf

,SHAPE_AREA,geometry,ECO_NAME
ecoregion,,,
0,0.038948,"MULTIPOLYGON (((158.7141 -69.60657, 158.71264 ...",Adelie Land tundra
1,0.170599,"MULTIPOLYGON (((147.28819 -2.57589, 147.2715 -...",Admiralty Islands lowland rain forests
2,13.844952,"MULTIPOLYGON (((26.88659 35.32161, 26.88297 35...",Aegean and Western Turkey sclerophyllous and m...
3,1.355536,"MULTIPOLYGON (((65.48655 34.71401, 65.52872 34...",Afghan Mountains semi-desert
4,8.196573,"MULTIPOLYGON (((-160.26404 58.64097, -160.2673...",Ahklun and Kilbuck Upland Tundra
...,...,...,...
842,9.422097,"MULTIPOLYGON (((117.33111 -7.53306, 117.30525 ...",Sulawesi lowland rain forests
843,5.010930,"MULTIPOLYGON (((36.7375 -3.13, 36.7375 -3.1316...",East African montane forests
844,0.890325,"MULTIPOLYGON (((36.38 -8.96583, 36.38 -8.96667...",Eastern Arc forests


In [4]:
gbif_ecoregion_gdf = (
    ecoregions_gdf
    # Match the CRS of the GBIF data and the ecoregions
    .to_crs(gbif_gdf.crs)
    # Find ecoregion for each observation
    .sjoin(
        gbif_gdf,
        how='inner', 
        predicate='intersects')
    # Select the required columns

)
gbif_ecoregion_gdf

,SHAPE_AREA,geometry,ECO_NAME,gbifID,month
ecoregion,,,,,
12,17.133639,"MULTIPOLYGON (((-119.53979 55.81661, -119.5443...",Alberta-British Columbia foothills forests,4843356029,6
12,17.133639,"MULTIPOLYGON (((-119.53979 55.81661, -119.5443...",Alberta-British Columbia foothills forests,4692720233,6
12,17.133639,"MULTIPOLYGON (((-119.53979 55.81661, -119.5443...",Alberta-British Columbia foothills forests,4697503370,6
12,17.133639,"MULTIPOLYGON (((-119.53979 55.81661, -119.5443...",Alberta-British Columbia foothills forests,4762874662,6
12,17.133639,"MULTIPOLYGON (((-119.53979 55.81661, -119.5443...",Alberta-British Columbia foothills forests,4832533846,6
...,...,...,...,...,...
833,35.905513,"POLYGON ((-119.99977 54.53117, -119.8914 54.45...",Northern Rockies conifer forests,4837678064,6
833,35.905513,"POLYGON ((-119.99977 54.53117, -119.8914 54.45...",Northern Rockies conifer forests,4758522155,6
833,35.905513,"POLYGON ((-119.99977 54.53117, -119.8914 54.45...",Northern Rockies conifer forests,4838904359,6


### Count the observations in each ecoregion each month

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Group observations by ecoregion</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Replace <code>columns_to_group_by</code> with a list of columns.
Keep in mind that you will end up with one row for each group – you want
to count the observations in each ecoregion by month.</li>
<li>Select only month/ecosystem combinations that have more than one
occurrence recorded, since a single occurrence could be an error.</li>
<li>Use the <code>.groupby()</code> and <code>.mean()</code> methods to
compute the mean occurrences by ecoregion and by month.</li>
<li>Run the code – it will normalize the number of occurrences by month
and ecoretion.</li>
</ol></div></div>

In [13]:
occurrence_df = (
    gbif_ecoregion_gdf
    # For each ecoregion, for each month...
    .groupby(['month', 'ecoregion'])
    # ...count the number of occurrences
    .agg(
        occurrences=('ECO_NAME', 'count'),
        area=('SHAPE_AREA', 'first'))
)

# Get rid of rare observations (possible misidentification?)
occurrence_df = occurrence_df[occurrence_df.occurrences>5]

#Create Area-Normalized column
occurrence_df['NormAreaOccurrences'] = (occurrence_df.occurrences / 
                                        occurrence_df.area)
occurrence_df = occurrence_df[['NormAreaOccurrences', 'occurrences']]

# Take the mean by ecoregion
mean_occurrences_by_ecoregion = (
    occurrence_df
    .groupby(['ecoregion'])
    .mean()
)
# Take the mean by month
mean_occurrences_by_month = (
    occurrence_df
    .groupby(['month'])
    .mean()
)

occurrence_df

NormAreaOccurrences  occurrences
month ecoregion                                  
1     43                    5.712587           62
      59                    6.891023           49
      73                    2.603209           21
      87                  454.132623         2147
      88                  457.402838         1451
...                              ...          ...
12    720                  16.713584          191
      729                  67.901765          280
      783                  33.211837          277
      790                  38.733471         1910
      793                 248.922141          422

[579 rows x 2 columns]

### Normalize the observations

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It: Normalize</div></div><div class="callout-body-container callout-body"><ol type="1">
<li>Divide occurrences by the mean occurrences by month AND the mean
occurrences by ecoregion</li>
</ol></div></div>

In [22]:
# Normalize by regional variation and time for sampling effort
normoccurences_df = (
    occurrence_df
    / occurrence_df
    / mean_occurrences_by_ecoregion
    / mean_occurrences_by_month
)
#Add non-normalized occurrences
normoccurences_df['total_occurrences'] = occurrence_df['occurrences'] 

#Select normalized and absolute occurences columns
normoccurences_df = normoccurences_df[['NormAreaOccurrences', 'total_occurrences']]

normoccurences_df

NormAreaOccurrences  total_occurrences
month ecoregion                                        
1     43                    0.001747                 62
      59                    0.005486                 49
      73                    0.000564                 21
      87                    0.000063               2147
      88                    0.000061               1451
...                              ...                ...
12    720                   0.001212                191
      729                   0.000343                280
      783                   0.000532                277
      790                   0.000207               1910
      793                   0.000068                422

[579 rows x 2 columns]

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Make sure to store the new version of your <code>DataFrame</code> for
other notebooks!</p>
<div id="015f18c7" class="cell" data-execution_count="9">
<div class="sourceCode" id="cb1"><pre
class="sourceCode python cell-code"><code class="sourceCode python"><span id="cb1-1"><a href="#cb1-1" aria-hidden="true" tabindex="-1"></a><span class="op">%</span>store occurrence_df</span></code></pre></div>
</div></div></div>

## Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! This code will store all your variables. You might want to
specify specific variables, especially if you have large objects in
memory that you won’t need in the future.

In [23]:
%store normoccurences_df

Stored 'normoccurences_df' (DataFrame)


Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!